# 哈希表的基本操作
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 哈希表（Hash）

### 基本功能

Redis的Hash被设计出来，就是为了存储大量的键值对映射。储存相同数量的键值映射，Hash所占用的内存空间，远远小于字符串。

### 基本语法

```python
import redis

client = redis.Redis()

# 向Hash表中添加一个键值对
client.hset('Key', '字段名', '值')

# 向Hash表中添加多个键值对
client.hmset('Key', {'字段名1': '值1', '字段名2': '值2', '字段名3': '值3'})

# 查询字段是否在哈希表中
client.hexists('Key', '字段名')

# 查询哈希表中一个有多少个字段
client.hlen('Key')

# 获取Hash表里面所有的字段名（慎用）
client.hkeys('Key')

# 读取一个字段中的数据
client.hget('Key', '字段名')

# 读取多个字段中的数据
client.hmget('Key', ['字段名1', '字段名2', '字段名3'])

# 读取全部字段（慎用）
client.hgetall('Key')
```

## 向哈希表中写入数据

In [ ]:
# 初始化Redis连接
import redis
import json
client = redis.Redis()

In [ ]:
# 添加一条数据

info = json.dumps({'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'})
client.hset('user', 10001, info)

In [ ]:
# 插入多条数据
info_dict = {
    10002: json.dumps({'name': '王小三', 'age': 27, 'salary': 10000, 'address': '浙江'}),
    10003: json.dumps({'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}),
    10004: json.dumps({'name': '刘小五', 'age': 30, 'salary': 990, 'address': '武汉'})
}
client.hmset('user', info_dict)

In [ ]:
# 字段名不一定非要是数字，也可以是字母或者中文，字段值的数据类型也可以任意设定

client.hset('user', '马小七', 780)

## 检查字段信息

In [ ]:
# 检查字段是否在Hash表中
client.hexists('user', 10003)

In [ ]:
client.hexists('user', '马小七')

In [ ]:
client.hexists('user', '不存在的字段')

In [ ]:
client.hexists('不存在的Key', 10003)

In [ ]:
# 查询Hash中有多少字段
client.hlen('user')

In [ ]:
keys = client.hkeys('user')
for key in keys:
    print(key.decode())

## 获取键值对


In [ ]:
# 获取单条数据
data = client.hget('user', '马小七')
print(f'返回的数据类型为：{type(data)}, 数据内容为：{data}')
print(f'数据解析以后为：{data.decode()}')


In [ ]:
data = client.hget('user', 10003)
data_dict = json.loads(data)
print(f'用JSON解析以后：{data_dict}')

In [ ]:
# 批量获取数据

data_list = client.hmget('user', [10001, 10003])
for data in data_list:
    print(json.loads(data))

In [ ]:
## 获取全部数据

all_data = client.hgetall('user')
print(f'先来看看返回的数据是什么样的：{all_data}')



In [ ]:
for field, value in all_data.items():
    print(f'字段名：{field.decode()}, 值：{json.loads(value)}')

![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)

## 使用列表和Hash实现简单的任务队列
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


### 为什么需要任务队列？

有一系列比较耗时的任务，不能实时完成，此时就需要使用任务队列排队完成。例如，网站注册时需要发送验证邮件。发送1封邮件需要2秒钟。现在只有一台邮件服务器，有100人同时注册。

发邮件的过程不能让网站来完成。网站只是创建发邮件的任务，并把任务扔进任务队列中。另一个专门负责发邮件的程序从任务队列中读取任务，然后执行具体的发送操作。

### 任务队列需要实现哪些功能？

1. 添加任务
2. 删除任务
3. 暂停任务
4. 恢复被暂停的任务并重新排队

### 如何设计邮件的数据结构？

* 列表中存放任务ID
* Hash中存放任务详情，字段名为任务ID

任务详情的结构为：

```python
{
    "task_id": 123,
    "target": "xxx@163.com",
    "created_time": "2019-03-24 11:12:34"
}
```

## 添加任务

* 哈希表的Key为：task:detail
* 列表的Key为：task:queue

1. 首先创建任务详情，并写入Hash表中
2. 把任务ID写入到列表中

In [1]:
# 初始化Redis连接

import redis
client = redis.Redis()

In [3]:
# 添加任务
import datetime
import json


def add_task(task_id, target):
    task_detail = {'task_id': task_id,
                   'target': target,
                   'created_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    client.hset('task:detail', task_id, json.dumps(task_detail))
    client.rpush('task:queue', task_id)

In [4]:
# 发邮件的程序读取任务

def read_task():
    task_id = client.blpop('task:queue')[1].decode()
    task_detail = client.hget('task:detail', task_id)
    target = json.loads(task_detail)['target']
    print(f'给：{target} 发送邮件')
    

In [5]:
# 删除任务

def del_task(task_id):
    client.lrem('task:queue', 0, task_id)
    client.hdel('task:detail', task_id)

## 暂停和恢复任务

暂停和恢复任务都不会影响task:detail，只需要控制task:queue中的task_id即可。

### 暂停任务

1. 把task_id从task:queue中移除
2. 把task_id放入暂停列表：task:pause中

### 恢复任务

1. 把task_id从task:pause中移除
2. 把task_id重新放入task:queue右侧

In [6]:
# 暂停任务

def pause_task(task_id):
    client.lrem('task:queue', 0, task_id)
    client.rpush('task:pause', task_id)

In [7]:
# 恢复任务

def resume_task(task_id):
    client.lrem('task:pause', 0, task_id)
    client.rpush('task:queue', task_id)

## 来测试一下我们的简易任务队列

In [8]:
# 添加任务
add_task(1, 'contact@kingname.info')

In [9]:
add_task(2, 'register@163.com')

In [10]:
# 读取任务
read_task()

给：contact@kingname.info 发送邮件


In [11]:
add_task(3, 'rain@gmail.com')

In [12]:
add_task(4, 'world@hotmail.com')

In [13]:
## 删除任务
del_task(2)

In [14]:
add_task(5, 'hello@facebook.com')

In [15]:
pause_task(4)

In [16]:
read_task()

给：rain@gmail.com 发送邮件


In [17]:
read_task()

给：hello@facebook.com 发送邮件


In [18]:
# 恢复任务
resume_task(4)

In [19]:
read_task()

给：world@hotmail.com 发送邮件


![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)